> **Tip**: Welcome to the Investigate a Dataset project! You will find tips in quoted sections like this to help organize your approach to your investigation. Before submitting your project, it will be a good idea to go back through your report and remove these sections to make the presentation of your work as tidy as possible. First things first, you might want to double-click this Markdown cell and change the title so that it reflects your dataset and investigation.

# Investigation of Kaggle's soccer database
## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> **Tip**: In this section of the report, provide a brief introduction to the dataset you've selected for analysis. At the end of this section, describe the questions that you plan on exploring over the course of the report. Try to build your report around the analysis of at least one dependent variable and three independent variables.
>
> If you haven't yet selected and downloaded your data, make sure you do that first before coming back here. If you're not sure what questions to ask right now, then make sure you familiarize yourself with the variables and the dataset context for ideas of what to explore.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as sqla
import xml.etree.ElementTree as ET
%matplotlib inline


<a id='wrangling'></a>
## Data Wrangling

> **Tip**: In this section of the report, you will load in the data, check for cleanliness, and then trim and clean your dataset for analysis. Make sure that you document your steps carefully and justify your cleaning decisions.

### General Properties

In [8]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.
# build sql query to extract match data - ignore data without valid possession data
query_match = """select match.id as match_id, league.name, season, possession, home_team_goal, away_team_goal, shoton, shotoff
from Match inner join league on match.league_id = league.id
where league_id = 1729
 """
query_player = """
select * from player;
 """
db = sqla.create_engine('sqlite:///data/database.sqlite')
df_match = pd.read_sql(query_match, db)
df_player = pd.read_sql(query_player,db)
print(df_match.iloc[:3,:3])
print(df_player.head())
print(df_match.dtypes)
print(df_player.dtypes)

   match_id                    name     season
0      1729  England Premier League  2008/2009
1      1730  England Premier League  2008/2009
2      1731  England Premier League  2008/2009
   id  player_api_id         player_name  player_fifa_api_id  \
0   1         505942  Aaron Appindangoye              218353   
1   2         155782     Aaron Cresswell              189615   
2   3         162549         Aaron Doran              186170   
3   4          30572       Aaron Galindo              140161   
4   5          23780        Aaron Hughes               17725   

              birthday  height  weight  
0  1992-02-29 00:00:00  182.88     187  
1  1989-12-15 00:00:00  170.18     146  
2  1991-05-13 00:00:00  170.18     163  
3  1982-05-08 00:00:00  182.88     198  
4  1979-11-08 00:00:00  182.88     154  
match_id           int64
name              object
season            object
possession        object
home_team_goal     int64
away_team_goal     int64
shoton            object
shotof

> **Tip**: You should _not_ perform too many operations in each cell. Create cells freely to explore your data. One option that you can take with this project is to do a lot of explorations in an initial notebook. These don't have to be organized, but make sure you use enough comments to understand the purpose of each code cell. Then, after you're done with your analysis, create a duplicate notebook where you will trim the excess and organize your steps so that you have a flowing, cohesive report.

> **Tip**: Make sure that you keep your reader informed on the steps that you are taking in your investigation. Follow every code cell, or every set of related code cells, with a markdown cell to describe to the reader what was found in the preceding cell(s). Try to make it so that the reader can then understand what they will be seeing in the following cell(s).

### Data Cleaning
First defining some helper functions, see the comments in the line above the definition to learn about their specific tasks
```
def flatten(list_of_list)

def build_possession_row_from_xml(xmlstring, matchid)

def build_shotlists_from_xml(xmlstring, matchid)
```



In [45]:
# Helper function flatten, flattening of lists. occassionally we have to deal with lists of lists
#which have to be flattened to be used in data frames.
def flatten(list_of_list):
    flattened_list = [y for x in list_of_list for y in x]
    return flattened_list

# Helper function to extract xml data from the column possession of data frame df_match
def build_possession_row_from_xml(xmlstring, matchid):
    root = ET.fromstring(xmlstring)
    # we are interested in the last entry only, usually the possession values after 90 minutes
    tags = root.findall("value[last()]")

    rows = []
    #there should be only one tag found, but leaving loop for possible later changes
    for node in tags:
        #print(node)
        elapsed = node.findtext("elapsed") if node is not None else None
        homepos = node.findtext("homepos") if node is not None else None
        
        awaypos = node.findtext("awaypos") if node is not None else None
        rows.append({"match_id": matchid, "elapsed": elapsed, "homepos": homepos, 
                     "awaypos": awaypos})
    if rows.count == 0:
        return  None
    else:
        #return value is a list of dictionaries
        return rows

# Helper function to extract xml data from the columns shotson and shotsoff of df_match
def build_shotlists_from_xml(xmlstring, matchid):
    root = ET.fromstring(xmlstring)
   
    rows = []
    for node in root: 
        elapsed = node.findtext("elapsed") if node is not None else None
        team = node.findtext("team") if node is not None else None
        type_s = node.findtext("type") if node is not None else None
        player = node.findtext("player1") if node is not None else None
        rows.append({"match_id": matchid, "elapsed": elapsed, "team": team, "player": player,
                     "type": type_s})
    if rows.count == 0:
        return  None
    else:
        return rows

> **create additional data frames**  
Creating some extra data frames from xml content of some columns of the `df_match` data frame with the code of next cells.  
These data frames are needed later for further analysis.

#### Create data frame `df_pos` for possession data

In [49]:
# parse xml data of column possession with helper funcion
rows_pos = df_match.apply(lambda x : build_possession_row_from_xml(x['possession'], x['match_id']), axis=1)
flattened_list = flatten(rows_pos.values.tolist())

# build new data frame with possession data and show a few lines and data types
df_pos = pd.DataFrame(flattened_list)
print(df_pos.loc[:3])
print(df_pos.dtypes)
# show possible NaN values
print(df_pos.isna().sum())

   match_id elapsed homepos awaypos
0      1729      90      55      45
1      1730      90      66      34
2      1731      90      46      54
3      1732      90      52      48
match_id     int64
elapsed     object
homepos     object
awaypos     object
dtype: object
match_id    0
elapsed     0
homepos     1
awaypos     1
dtype: int64


##### comment
> 3 columns of new data frame `df_pos` are all strings because of the previous xml parsing.  
So, casting columns to ints. *homepos, awaypos* both have NaNs in certain rows. We have to drop affected rows first.



In [50]:
# get rid of rows with NaNs in homepos or awaypos
df_pos.dropna(subset=['homepos','awaypos'], inplace=True)
# set all columns to int64 and check with dtype
df_pos = df_pos.astype('int64')
df_pos.dtypes



match_id    int64
elapsed     int64
homepos     int64
awaypos     int64
dtype: object

#### Create data frame `df_shots` from shotson and shotsoff data

In [80]:
rows_on = df_match.apply(lambda x : build_shotlists_from_xml(x['shoton'], x['match_id']), axis=1)
df_shoton = pd.DataFrame(flatten(rows_on.values.tolist()))
rows_off = df_match.apply(lambda x : build_shotlists_from_xml(x['shotoff'], x['match_id']), axis=1)
df_shotoff = pd.DataFrame(flatten(rows_off.values.tolist()))
df_shots = pd.concat([df_shotoff, df_shoton])
df_shots[df_shots['match_id']==1729].sort_values(by=['elapsed'])
df_shots['player'] = pd.to_numeric(df_shots['player'], downcast='integer', errors='coerce')
#df_shots['player'] = df_shots[df_shots['player'].notnull()]['player'].astype('int64')
df_shots.dtypes

match_id      int64
elapsed      object
team         object
player      float64
type         object
dtype: object

In [82]:
result = df_player.merge(df_shots, left_on=('id'), right_on=('player'))
result.groupby('player_name').size().sort_values()

player_name
Franck Jurietti            7
Tugay Kerimoglou          59
Ricky van Wolfswinkel     63
Dudu                      70
Franco Di Santo          189
Ricardo Oliveira         227
dtype: int64

<a id='eda'></a>
## Exploratory Data Analysis

> **Tip**: Now that you've trimmed and cleaned your data, you're ready to move on to exploration. Compute statistics and create visualizations with the goal of addressing the research questions that you posed in the Introduction section. It is recommended that you be systematic with your approach. Look at one variable at a time, and then follow it up by looking at relationships between variables.

### Research Question 1 (Replace this header name!)

In [ ]:
# Use this, and more code cells, to explore your data. Don't forget to add
#   Markdown cells to document your observations and findings.


### Research Question 2  (Replace this header name!)

In [ ]:
# Continue to explore the data to address your additional research
#   questions. Add more headers as needed if you have more questions to
#   investigate.


<a id='conclusions'></a>
## Conclusions

> **Tip**: Finally, summarize your findings and the results that have been performed. Make sure that you are clear with regards to the limitations of your exploration. If you haven't done any statistical tests, do not imply any statistical conclusions. And make sure you avoid implying causation from correlation!

> **Tip**: Once you are satisfied with your work, you should save a copy of the report in HTML or PDF form via the **File** > **Download as** submenu. Before exporting your report, check over it to make sure that the flow of the report is complete. You should probably remove all of the "Tip" quotes like this one so that the presentation is as tidy as possible. Congratulations!